In [ ]:
using JuMP, SDDP, HiGHS, Gurobi, Statistics, Test, DataStructures

# Performance of SDDP

In [ ]:
# In Julia this struct is immutable
struct HyperParams
    tau             ::Int64
    nOrigins        ::Int64
    nDestinations   ::Int64
    nLanes          ::Int64
    nCarriers       ::Int64
    Bids            ::Vector{Vector{Int64}}
    Winners         ::OrderedDict{Int64, Vector{Int64}}
    CarrierIdx      ::Vector{Vector{Int64}}
    from_           ::Vector{Vector{Int64}}
    to_             ::Vector{Vector{Int64}}
    entry_stock_0   ::Vector{Int64}
    exit_stock_0    ::Vector{Int64}
    exit_short_0    ::Vector{Int64}
    entry_capacity  ::Vector{Int64}
    exit_capacity   ::Vector{Int64}
    flow_support    ::Vector{NTuple}
    entry_store_coef::Vector{Float64}
    exit_store_coef ::Vector{Float64}
    exit_short_coef ::Vector{Float64}
    transport_coef  ::Vector{Float64}
    carrier_capacity::Vector{Int64}
end

# Outer constructor with arguments
function HyperParams(; 
                     tau             ::Int64 = 5,
                     nOrigins        ::Int64 = 2,
                     nDestinations   ::Int64 = 2,
                     nCarriers       ::Int64 = 3,
                     Bids            ::Vector{Vector{Int64}} = [[1, 2, 3, 4],
                                                                [1, 4],
                                                                [2, 3],
                                                                [1, 3, 4],
                                                                [2, 4]],
                     Winners         ::OrderedDict{Int64, Vector{Int64}} = OrderedDict(
                                                                             1 => [2, 3], 
                                                                             2 => [4, 5], 
                                                                             3 => [1]),
                     entry_stock_0   ::Vector{Int64} = [0, 0],
                     exit_stock_0    ::Vector{Int64} = [22, 0],
                     exit_short_0    ::Vector{Int64} = [0, 0],
                     entry_capacity  ::Vector{Int64} = [1000, 1000],
                     exit_capacity   ::Vector{Int64} = [1000, 1000],
                     flow_support    ::Vector{Int64} = [0, 100, 300, 500],
                     entry_store_coef::Vector{Float64} = [2.0, 2.0],
                     exit_store_coef ::Vector{Float64} = [1.0, 1.0],
                     exit_short_coef ::Vector{Float64} = [3.0, 3.0],
                     transport_coef  ::Vector{Float64} = [ 4.8, 3.3, 6, 6.9, 7.3, 5.9, 5.8, 5.4, 3.3, 6.4, 6.9, 5.2, 6.3 ], 
                     carrier_capacity::Vector{Int64} = [160, 100, 140, 120, 100,  # Carrier 1
                                                        140, 100, 160, 180, 100,  # Carrier 2
                                                        300, 300, 300, 300, 300]) # Carrier 3

    # nLanes = nOrigins * nDestinations

    from_ = Dict{Int64, Vector{Int64}}()
    for i in 1:nOrigins
        from_[i] = (collect(1:nDestinations) .- 1) .* nOrigins .+ i
    end

    to_ = Dict{Int64, Vector{Int64}}()
    for j in 1:nDestinations
        to_[j] = (j .- 1) .* nOrigins .+ collect(1:nOrigins)
    end

    ordx = vcat(collect(values(Winners))...)
    Ldx  = vcat(Bids[ordx]...)

    nLc = [0; cumsum([ length(vcat(Bids[Winners[cdx]]...)) for cdx in 1:nCarriers ])]

    HyperParams(
        tau,
        nOrigins,
        nDestinations,
        nLc[end], # nLanes
        nCarriers,
        Bids,
        Winners,
        [ collect(nLc[i] + 1 : nLc[i + 1]) for i in 1:length(nLc) - 1 ],   # CarrierIdx
        [ findall(lane -> lane in from_[i], Ldx) for i in 1:nOrigins ],    # from_
        [ findall(lane -> lane in to_[j], Ldx) for j in 1:nDestinations ], # to_
        entry_stock_0,
        exit_stock_0,
        exit_short_0,
        entry_capacity,
        exit_capacity,
        vec(collect(Base.product(ntuple(_ -> flow_support, nOrigins + nDestinations)...))), # Exhaustive 
        entry_store_coef,
        exit_store_coef,
        exit_short_coef,
        transport_coef,
        carrier_capacity)
end

In [ ]:
function transportation_t(sp::Model, stage::Int64; config::HyperParams)
    ## State variables
    @variables(sp, begin
        0 <= entry[i = 1:config.nOrigins],      (SDDP.State, initial_value = config.entry_stock_0[i])
        0 <= exitp[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_stock_0[j])
        0 <= exitm[j = 1:config.nDestinations], (SDDP.State, initial_value = config.exit_short_0[j])
        0 <= move[k  = 1:config.nLanes]
        inflow[ i = 1:config.nOrigins]
        outflow[j = 1:config.nDestinations]
    end)
    ## Decision variables
    ## Constraints
    # Carrier capacity
    @constraint(sp, [k = 1:config.nCarriers], sum(move[config.CarrierIdx[k]]) <= config.carrier_capacity[(k - 1) * config.tau + stage])
    # Entry volume
    @constraint(sp, [i = 1:config.nOrigins], sum(move[config.from_[i]]) <= entry[i].in + inflow[i])
    # Storage limit
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].in + sum(move[config.to_[j]]) <= config.exit_capacity[j])
    # Transition
    @constraint(sp, [i = 1:config.nOrigins],      entry[i].out == entry[i].in + inflow[i] - sum(move[config.from_[i]]))
    @constraint(sp, [j = 1:config.nDestinations], exitp[j].out - exitm[j].out == exitp[j].in - exitm[j].in + sum(move[config.to_[j]]) - outflow[j])
    # Uncertain variables
    # Ξ = config.flow_support
    Ξ = config.flow_support[rand(1:length(config.flow_support), 1000)]
    SDDP.parameterize(sp, Ξ) do ξ
        JuMP.fix.(inflow,  ξ[1:config.nOrigins])
        JuMP.fix.(outflow, ξ[config.nOrigins .+ (1:config.nDestinations)])
    end
    ## Objective function
    @stageobjective(sp, 
        sum(config.entry_store_coef[i] * entry[i].in for i in 1:config.nOrigins)      + 
        sum(config.exit_store_coef[j]  * exitp[j].in for j in 1:config.nDestinations) + 
        sum(config.exit_short_coef[j]  * exitm[j].in for j in 1:config.nDestinations) + 
        sum(sum(config.transport_coef[config.CarrierIdx[k]] .* move[config.CarrierIdx[k]]) for k in 1:config.nCarriers)
    )
end

In [ ]:
# Hyperparameter configuration
config = HyperParams(
    tau             = 4,
    nOrigins        = 4,
    nDestinations   = 4,
    nCarriers       = 3,
    Bids            = [[1, 2, 3, 4, 5, 6, 7, 8],
                       [5, 6, 7, 8, 9, 10, 11, 12],
                       [10, 11, 12, 13, 14, 15, 16],
                       [1, 2, 3, 4, 8, 9, 10, 11],
                       [5, 6, 7, 8, 13, 14, 15, 16]],
    entry_stock_0   = [500, 500, 500, 500],
    exit_stock_0    = [0, 0, 0, 0],
    exit_short_0    = [0, 0, 0, 0],
    entry_capacity  = [1000, 1000, 1000, 1000],
    exit_capacity   = [1000, 1000, 1000, 1000],
    flow_support    = [0, 100, 300, 500],
    entry_store_coef= [2.0, 2.0, 2.0, 2.0],
    exit_store_coef = [1.0, 1.0, 1.0, 1.0],
    exit_short_coef = [3.0, 3.0, 3.0, 3.0],
    transport_coef  = [ 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3, 2.0, 1.7, 2.1, 3.3,   # Carrier 1 / Lane 1 - 16
                        1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7, 1.7, 1.9, 2.6, 3.7,   # Carrier 2 / Lane 1 - 16
                        2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1, 2.4, 1.2, 2.9, 3.1 ], # Carrier 3 / Lane 1 - 16
    carrier_capacity= [160, 100, 140, 120,  # Carrier 1 / Time Stage 1 - 4
                       140, 100, 160, 180,  # Carrier 2 / Time Stage 1 - 4
                       300, 300, 300, 300 ] # Carrier 3 / Time Stage 1 - 4
);

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 200,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

# Scaling to realistic instances

In [ ]:
using Distributions

In [ ]:
# Hyperparameter configuration
config = HyperParams(
    tau             = 12,
    nOrigins        = 6,
    nDestinations   = 6,
    nCarriers       = 20,
    Bids            = [[1, 2, 3, 4, 5, 6, 7, 8],
                       [5, 6, 7, 8, 9, 10, 11, 12],
                       [10, 11, 12, 13, 14, 15, 16],
                       [14, 15, 16, 17, 18, 19, 20, 21],
                       [22, 23, 24, 25, 26, 27, 28, 29, 30],
                       [31, 32, 33, 34, 35, 36], 
                       [17, 18, 19, 20, 21, 22, 23, 24, 25], 
                       [22, 23, 24, 25, 26], 
                       [27, 28, 29, 30, 31, 32, 33, 34] 
                      ],
    Winners         = OrderedDict(
                         1 => [1, 2], 
                         2 => [3, 4], 
                         3 => [5], 
                         4 => [6, 7], 
                         5 => [8], 
                         6 => [6, 8], 
                         7 => [2, 3], 
                         8 => [4], 
                         9 => [9], 
                        10 => [1], 
                        11 => [5], 
                        12 => [7, 8], 
                        13 => [6], 
                        14 => [2], 
                        15 => [5, 8], 
                        16 => [9], 
                        17 => [2, 3], 
                        18 => [4, 7], 
                        19 => [3, 5], 
                        20 => [1] ),
    entry_stock_0   = [200, 0, 100, 0, 250, 200],
    exit_stock_0    = [100, 0, 0, 200, 100, 100],
    exit_short_0    = [0, 0, 0, 0, 0, 0],
    entry_capacity  = [5000, 5000, 5000, 5000, 5000, 5000],
    exit_capacity   = [5000, 5000, 5000, 5000, 5000, 5000],
    flow_support    = [0, 250, 500, 750],
    entry_store_coef= [2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
    exit_store_coef = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
    exit_short_coef = [3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
    transport_coef  = map(x -> round(x, digits = 2), rand(Uniform(4.0, 8.0), 225)), # Carrier 3 / Lane 1 - 16
    carrier_capacity= rand(30:70, 20 * 12)
);

In [ ]:
model = SDDP.PolicyGraph(
    (sp, stage) -> transportation_t(sp, stage; config = config),
    SDDP.LinearGraph(config.tau);
    sense       = :Min,
    lower_bound = 0.0,
    optimizer   = Gurobi.Optimizer,
)

In [ ]:
SDDP.train(model;
    iteration_limit = 200,
    cut_type        = SDDP.SINGLE_CUT,
    parallel_scheme = SDDP.Serial(),
)

# Sensitivity analysis

# Realistic uncertainty model

# Spot market uncertainty